In [389]:
#### Imports ####
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from matplotlib.pyplot import imshow, show, subplot,title,axis
from matplotlib.patches import Circle
from skimage.io import imread
import skimage.util
import skimage as sk
from timeit import timeit
from skimage.feature import corner_peaks,peak_local_max
import matplotlib.patches as patches
from matplotlib.colors import LinearSegmentedColormap
import os
from skimage import exposure, filters, morphology
from cellpose import models, io, denoise, utils
from cellpose.io import imread, logger_setup
import pandas as pd
from skimage.measure import regionprops, label, regionprops_table
from skimage.feature import blob_log
from skimage.exposure import rescale_intensity
from skimage import transform
from skimage.draw import polygon_perimeter
from scipy.ndimage import binary_erosion

In [390]:
def compute_cdf(histogram):
    # Compute the cumulative sum of the histogram
    cumulative_histogram = np.cumsum(histogram)
    # Normalize the cumulative histogram to obtain the CDF
    cdf = cumulative_histogram / np.sum(histogram)
    return cdf

def C(image, cdf):
    c = cdf[image]
    return c

def custom_colormap():
    n_intervals = 10
    intervals = np.linspace(0, 1, n_intervals)
    cmap_dict = {'red': [], 'green': [], 'blue': []}
    for interval in intervals:
        r = 1  # Max intensity (red)
        g = 0  # Min intensity (green)
        b = 0  # Min intensity (blue)
        cmap_dict['red'].append((interval, r, r))
        cmap_dict['green'].append((interval, g, g))
        cmap_dict['blue'].append((interval, b, b))
    custom_cmap = LinearSegmentedColormap('custom', cmap_dict)
    return custom_cmap

def detect_outline(image):
    
    image = image.astype(bool)
    struct = np.ones((3, 3), dtype=bool)
    eroded_image = binary_erosion(image, structure=struct)
    outline = image ^ eroded_image
    
    return outline

In [391]:
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        img = imread(image_path)
        if img is not None:
            images.append(img)
        else:
            print(f"Error loading image {filename}")
    return images

paths = [r"C:\Users\George\Desktop\Final_Project\Images\Control", r"C:\Users\George\Desktop\Final_Project\Images\Penetramax"]

In [392]:
output_folder = r'C:\Users\George\Desktop\Final_Project'

# Ensure the folder exists (create it if it doesn't)
# os.makedirs(output_folder, exist_ok=True)

# Define the filename for the CSV
output_filename = 'Segmented_Nuclei_train.csv'

In [393]:
io.logger_setup()

all_props = []
Nuclei = []

for path in paths:
    Images = load_images_from_folder(path)
    Rotated_90 = [transform.rotate(image, 90, resize=False) for image in Images]
    # Rotated_180 = [transform.rotate(image, 180, resize=False) for image in Images]
    # Rotated_270 = [transform.rotate(image, 270, resize=False) for image in Images]
    # Flipped_Horizontal = [image[:, ::-1] for image in Images]
    # Flipped_Vertical = [image[::-1, :] for image in Images]

    Images.extend(Rotated_90)
    # Images.extend(Rotated_180)
    # Images.extend(Rotated_270)
    # Images.extend(Flipped_Horizontal)
    # Images.extend(Flipped_Vertical)

    #Model without and with denoising
    # model = models.Cellpose(model_type='nuclei', gpu=True)
    model = denoise.CellposeDenoiseModel(gpu=True, model_type="nuclei", restore_type='denoise_nuclei')
    
    for i in range(len(Images)):
        #1.Isolating the red channel for each Image
        Red_channel = np.copy(Images[i][:, :, 0])
        # 2.1 Apply Histogram equalization 
        # hist, edges = np.histogram(Red_channel.flatten(), bins = 512, range = [0, 512])
        # CDF = compute_cdf(hist)
        # Red_channel = C(Red_channel, CDF)*255
        #2.2 Apply Adaptive Histogram equalization (play with kernal size and clip_limit), works better here
        Red_channel = skimage.exposure.equalize_adapthist(Red_channel, kernel_size=10, clip_limit=0.02, nbins=512)

        #3.Convert to binary using otsu
        threshold = filters.threshold_otsu(Red_channel)
        binary_image = (Red_channel > threshold).astype(int)
        binary_image = Red_channel

        #5.Segmentation
        masks, flows, styles, diams = model.eval(Red_channel, diameter=60, channels=[0, 0])
    
        # fig, axes = plt.subplots(1, 3, figsize=(20, 6))
        # ax = axes.ravel()

        # ax[0].imshow(Images[i])
        # ax[0].set_title('Original Image')
        # ax[1].imshow(Red_channel, cmap='gray')
        # ax[1].set_title('Brightened Image')
        # ax[2].imshow(Images[i])
        # ax[2].imshow(masks, alpha = 0.9*(masks>0.01), cmap = custom_colormap())
        # ax[2].set_title('Segmented Image')

        # for a in ax:
        #     a.axis('off')

        # plt.suptitle(f'Image {i + 1}', x = 0.5, y = 0.97, fontsize = 25)
        # plt.tight_layout()
        # plt.show()

        props = regionprops(masks, Red_channel, spacing=(1, 1))
        for j, prop in enumerate(props):

            min_row, min_col, max_row, max_col = prop['bbox']
            mask = prop.image
            New_Red = Images[i][min_row:max_row, min_col:max_col, 0] * mask
            Normalized_Red = rescale_intensity(New_Red, in_range='image', out_range=(0, 1))  
            Normalized_Red = filters.gaussian(Normalized_Red, sigma=1)
            # threshold = filters.threshold_otsu(Normalized_Red)
            threshold = 0.25
            New_binary_image = np.clip((Normalized_Red > threshold).astype(int) + detect_outline(mask), 0, 1)
            
            ###VISUALIZATION OF THE NUCLEI###
            # if path == paths[0]:
            #     fig, ax = plt.subplots(1, 3, figsize = (8,6))
    
            #     ax[0].imshow(mask, cmap = 'gray')
            #     ax[0].set_title('Mask', fontsize = 14)
            #     ax[1].imshow(Normalized_Red, cmap = 'gray')
            #     ax[1].set_title('Intensity image (Red channel)', fontsize = 14)
            #     ax[2].imshow(New_binary_image, cmap = 'gray')
            #     ax[2].set_title('Binary nucleus image', fontsize = 14)
            #     plt.tight_layout()
            #     plt.show()
                
            def Category(regionmask):
                category_value = 0 if path == paths[0] else 1
                return category_value

            table = regionprops_table(New_binary_image, Normalized_Red, properties = ['area', 'area_filled', 'area_convex', 'euler_number', 'mean_intensity',
            'solidity'],  extra_properties=(Category, ))
            
            area_perc = 100 - (table['area_filled'] - table['area']) / table['area_filled']
            table['area_percentage'] = area_perc

            for k in range(len(table['euler_number'])):  # Iterate through each region in the table
                row = {key: table[key][k] for key in table}
                all_props.append(row)
            Nuclei.append(Normalized_Red)

df = pd.DataFrame(all_props)
df.to_csv(output_filename, index=False)

creating new log file
2024-05-29 20:12:20,685 [INFO] WRITING LOG OUTPUT TO C:\Users\George\.cellpose\run.log
2024-05-29 20:12:20,686 [INFO] 
cellpose version: 	3.0.8 
platform:       	win32 
python version: 	3.11.4 
torch version:  	2.2.2+cu118
2024-05-29 20:12:22,339 [INFO] >> denoise_nuclei << model set to be used
2024-05-29 20:12:22,339 [INFO] ** TORCH CUDA version installed and working. **
2024-05-29 20:12:22,345 [INFO] >>>> using GPU
2024-05-29 20:12:22,564 [INFO] >>>> model diam_mean =  17.000 (ROIs rescaled to this size during training)
2024-05-29 20:12:22,564 [INFO] >> nuclei << model set to be used
2024-05-29 20:12:22,586 [INFO] ** TORCH CUDA version installed and working. **
2024-05-29 20:12:22,588 [INFO] >>>> using GPU
2024-05-29 20:12:22,696 [INFO] >>>> loading model C:\Users\George\.cellpose\models\nucleitorch_0
2024-05-29 20:12:22,846 [INFO] >>>> model diam_mean =  17.000 (ROIs rescaled to this size during training)


KeyboardInterrupt: 

In [ ]:
# N = 1290
# ira = np.zeros((Nuclei[N].shape[0], Nuclei[N].shape[1], 3))
# ira[:, :, 0], ira[:, :, 1], ira[:, :, 2] = Nuclei[N], 0, 0
# imshow(ira)
# ####Note: mean_intensity is based on the intensity image of regionprops####